# Trabalho TP3 Ex3
## Grupo 27
### LCC 2024/2025
#### Rafaela Antunes Pereira A102527
#### Gonçalo Gonçalves Barroso A102931
#### Ricardo Eusébio Cerqueira A102878

### Explicação do problema
Neste trabalho, pretende-se que tenho em conta o primeiro problema do TP2 relativo À descrição da cifra A5/1, ignorando a componente de geração final da chave.Antes de começar, identificamos que o SFOTS é baseado em três registos de deslocamento de comprimento fixo: X0, X1 e X2, com tamanhos de 19, 22 e 23 bits, respetivamente.

A condição inicial é dada por:

$$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad$$
A condição de erro (estado inválido) é o complemento da inicial:


 $$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$

### a)Codifique em “z3”  o SFOTS assim definido.


In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from z3 import *


 # Usando Z3 diretamente para interpolação
import itertools

 Iniciamos com a execução de uma função chamada `declare`, cujo objetivo é declarar as variáveis que representam o estado do sistema num dado instante \(i\). Cada registo é modelado como um vetor de bits, o que permite capturar de forma precisa o comportamento e os valores binários associados ao sistema nesse instante específico.

In [2]:
def declare(i):
    
      return {
        'X0': BitVec(f'X0_{i}', 19),
        'X1': BitVec(f'X1_{i}', 22),
        'X2': BitVec(f'X2_{i}', 23)
    }

Posteriormente, definiram-se as condições inicial e de erro. 

A condição inicial é satisfeita apenas se todos os registos tiverem valores positivos. Já a condição de erro ocorre sempre que a condição inicial não é satisfeita, ou seja, quando pelo menos um dos registos apresenta um valor que não seja positivo.

In [3]:
def init(state):
    return And(state['X0'] > 0,state['X1'] > 0,state['X2'] > 0)
def error(state):
    return Not(init(state))

De seguida  a função trans, que descreve a transição entre dois estados em um sistema de registros. A função utiliza operações bit a bit para calcular a transição entre os estados atuais (curr) e os estados próximos (prox) para três registradores (X0, X1 e X2)

In [4]:
def trans(atual, proximo):
    # Extrair bits de controlo
    controlo_1 = Extract(8, 8, atual['X0'])
    controlo_2 = Extract(10, 10, atual['X1'])
    controlo_3 = Extract(10, 10, atual['X2'])

    # Condição de maioria
    maioria = Or(controlo_1 == controlo_2, controlo_1 == controlo_3, controlo_2 == controlo_3)

    # Transição para X0
    feedback_X0 = Extract(18, 18, atual['X0']) ^ Extract(17, 17, atual['X0']) ^ Extract(16, 16, atual['X0']) ^ Extract(13, 13, atual['X0'])
    atualizacao_X0 = Concat(feedback_X0, Extract(18, 1, atual['X0']))
    t_X0 = If(maioria, proximo['X0'] == atualizacao_X0, proximo['X0'] == atual['X0'])

    # Transição para X1
    feedback_X1 = Extract(21, 21, atual['X1']) ^ Extract(20, 20, atual['X1'])
    atualizacao_X1 = Concat(feedback_X1, Extract(21, 1, atual['X1']))
    t_X1 = If(maioria, proximo['X1'] == atualizacao_X1, proximo['X1'] == atual['X1'])

    # Transição para X2
    feedback_X2 = Extract(22, 22, atual['X2']) ^ Extract(21, 21, atual['X2']) ^ Extract(20, 20, atual['X2']) ^ Extract(7, 7, atual['X2'])
    atualizacao_X2 = Concat(feedback_X2, Extract(22, 1, atual['X2']))
    t_X2 = If(maioria, proximo['X2'] == atualizacao_X2, proximo['X2'] == atual['X2'])

    # Retornar a condição final
    return And(t_X0, t_X1, t_X2)


Por fim, implementamos uma função chamada `gera_traco` que utiliza o solver Z3 para verificar a evolução de um sistema de transições entre estados. O objetivo da função é determinar se é possível ou não alcançar um estado de erro ao longo de uma sequência de transições. Caso o estado de erro não seja alcançado, a função imprime os valores dos registradores (LFSRs) em cada estado.

In [5]:
def gera_traco(init, error, trans, k):
    states = [declare(i) for i in range(k)]
    solver = Solver()

    # Adicionar a condição inicial
    solver.add(init(states[0]))

    # Garantir que nenhum estado seja de erro
    for i in range(k):
        solver.add(Not(error(states[i])))

    # Adicionar transições entre estados
    for i in range(k-1):
        solver.add(trans(states[i], states[i+1]))

    
    if solver.check() == sat:
        print("Estado de erro não é alcançável")
        model = solver.model()
        for i in range(k):
            print(f"\nEstado {i + 1}:")
            LFSR1val = model.evaluate(states[i]['X0']).as_long()
            LFSR2val = model.evaluate(states[i]['X1']).as_long()
            LFSR3val = model.evaluate(states[i]['X2']).as_long()
            print(f"LFSR 0 = {bin(LFSR1val)[2:].zfill(19)}")  # Formato de 19 bits
            print(f"LFSR 1 = {bin(LFSR2val)[2:].zfill(22)}")  # Formato de 22 bits
            print(f"LFSR 2 = {bin(LFSR3val)[2:].zfill(23)}")  # Formato de 23 bits
    else:
        print("Estado de erro é alcançável")




In [6]:

# Executar a função com 10 passos
gera_traco(init, error, trans, 10)


Estado de erro não é alcançável

Estado 1:
LFSR 0 = 0000001011100110000
LFSR 1 = 0001010100100100001111
LFSR 2 = 01001100000000110000101

Estado 2:
LFSR 0 = 0000000101110011000
LFSR 1 = 0000101010010010000111
LFSR 2 = 00100110000000011000010

Estado 3:
LFSR 0 = 0000000010111001100
LFSR 1 = 0000010101001001000011
LFSR 2 = 00010011000000001100001

Estado 4:
LFSR 0 = 0000000001011100110
LFSR 1 = 0000001010100100100001
LFSR 2 = 00001001100000000110000

Estado 5:
LFSR 0 = 0000000000101110011
LFSR 1 = 0000000101010010010000
LFSR 2 = 00000100110000000011000

Estado 6:
LFSR 0 = 0000000000010111001
LFSR 1 = 0000000010101001001000
LFSR 2 = 00000010011000000001100

Estado 7:
LFSR 0 = 0000000000001011100
LFSR 1 = 0000000001010100100100
LFSR 2 = 00000001001100000000110

Estado 8:
LFSR 0 = 0000000000000101110
LFSR 1 = 0000000000101010010010
LFSR 2 = 00000000100110000000011

Estado 9:
LFSR 0 = 0000000000000010111
LFSR 1 = 0000000000010101001001
LFSR 2 = 00000000010011000000001

Estado 10:
LFSR 0 = 00

### b) Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.
 A função estados_inseguros foi criada para verificar se existe algum estado que viole a propriedade de segurança. Se um estado inseguro for encontrado, ele é devolvido para que possa ser analisado ou bloqueado em iterações subsequentes do algoritmo PDR.

In [7]:
def estados_inseguros(frame_atual, erro, estado_atual, solver):   
    estado = declare(estado_atual)
    solver.push()
    solver.add(Not(frame_atual))  
    solver.add(erro(estado))      
    
    if solver.check() == sat:
        modelo = solver.model()
        estado_inseguro = {variavel: modelo.eval(variavel, model_completion=True) for variavel in estado.values()}
        solver.pop()
        return estado_inseguro

    solver.pop()
    return None

A função bloquear_inseguros tem como objetivo impedir que estados inseguros sejam alcançados. Ela faz isso adicionando restrições ao solver para bloquear esses estados, garantindo que o sistema não possa atingir um estado inválido durante as iterações. Quando um estado inseguro é bloqueado, a função retorna True, indicando que o bloqueio foi bem-sucedido.

In [8]:
def bloquear_inseguros(estado_inseguro, frames, transicao, solver):
    for indice_frame in range(len(frames) - 1, 0, -1):
        solver.push()
        estado_anterior = declare(indice_frame - 1)
        estado_seguinte = declare(indice_frame)

       
        solver.add(transicao(estado_anterior, estado_seguinte))
        solver.add(Not(frames[indice_frame - 1]))

        
        restricao_cubo = And(*(variavel != valor for variavel, valor in estado_inseguro.items()))
        solver.add(restricao_cubo)

        if solver.check() == unsat:
            solver.pop()
            
            frames[indice_frame] = And(frames[indice_frame], Not(restricao_cubo))
            print(f"Estado bloqueado no frame {indice_frame}")
            return True

        solver.pop()

    return False

A função PDR implementa o algoritmo Property Directed Reachability (PDR), usado para verificar a segurança de sistemas. Ela analisa iterativamente se existem estados inseguros, começando do estado inicial e considerando as transições. Se um estado inseguro for encontrado, tenta bloqueá-lo. Caso todos os estados inseguros sejam bloqueados ou não existam, conclui que o sistema é seguro; caso contrário, indica que a propriedade de segurança foi violada. O processo continua até verificar a propriedade ou identificar uma violação.

In [9]:


def PDR(inicial, transicao, erro):
   
    solver = Solver()
    frames = [Not(inicial(declare(0)))]
    iteracao = 0

    while True:
        print(f"Iteração {iteracao}")

        estado_critico = estados_inseguros(frames[iteracao], erro, iteracao, solver)
        if estado_critico:
            if not bloquear_inseguros(estado_critico, frames, transicao, solver):
                print("Propriedade violada: sistema inseguro!")
                return False
        else:
            if iteracao > 0 and frames[iteracao] == frames[iteracao - 1]:
                print("Propriedade verificada: sistema seguro!")
                return True

            frames.append(True)
            iteracao += 1



In [10]:
PDR(init,trans,error)

Iteração 0
Iteração 1
Iteração 2
Propriedade verificada: sistema seguro!


True